In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [43]:
def sklearn_model(X_train, t_train, X_test, t_test):

    model = LogisticRegression(random_state=0).fit(X_train, t_train)
    params = model.coef_[0]
    print(params)
    print(model.score(X_test, t_test))

In [5]:
def normalize(data, method='min-max'):
    
    """
    Computes statistics assuming each sample is a row.
    """
    
    if (method == 'min-max'):
        numerator = data - np.min(data, axis=0)
        denominator = np.max(data, axis=0) - np.min(data, axis=0)
        return numerator/denominator
    
    if (method == 'standardization'):
        numerator = data - np.mean(data, axis=0)
        denominator = np.std(data, axis=0)
        return numerator/denominator
    

In [6]:
def segregate_target(data):
    
    X = data[:, :-1]
    t = data[:, -1:]
    
    return X, t

In [75]:
def train_test_validation_split(X, t):
    
    X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.4, random_state=42)
    X_valid, X_test, t_valid, t_test = train_test_split(X_test, t_test, test_size=0.5, random_state=42)
    
    data = {
        'X_train': X_train,
        't_train': t_train,
        'X_valid': X_valid,
        't_valid': t_valid,
        'X_test': X_test,
        't_test': t_test
    }
    
    return data

In [8]:
def load_data(path):
    
    data = np.load(path)
    return data

In [9]:
def initialize_parameters(num, random=True):
    
    if random:
        w = np.random.randn(num).reshape(-1, 1)
    else:
        w = np.zeros((num, 1))
        
    b = 0
    
    return w, b

In [45]:
def sigmoid(x):
    
    return (1 / (1 + np.exp(-x)))

In [92]:
def compute_error(y, t):
    
    y = np.where(y >= 1, 0.99, y)
    m = y.shape[1]
    for i in range(m):
        if y[0][i] <= 0 or y[0][i] >= 1:
            print(i)
    return (1/m)*-np.sum(np.multiply(t, np.log(y)) + np.multiply(1 - t, np.log(1 - y)))

In [47]:
def propagate(X, w, b, t):
    
    m = X.shape[1]
    
    # forward calculation
    y = sigmoid(np.dot(w.T, X) + b)
    
    # cost calculation
    cost = compute_error(y, t)
    
    # backward calculation
    dw = (1/m)*np.dot(X, (y - t).T)
    db = (1/m)*np.sum(y - t)
    
    grads = {'dw': dw, 'db': db}
    
    return grads, cost

In [48]:
def update(w, b, grads, learningRate=0.001):
    
    dw = grads['dw']
    db = grads['db']
    
    w = w - learningRate*dw
    b = b - learningRate*db
    
    return w, b

In [54]:
def predict(w, b, X):
    
    M = X.shape[1]
    probabilities = sigmoid(np.dot(w.T, X) + b)
    predictions = np.zeros((1, M))
    
    for i in range(M):
        predictions[0][i] = 0 if probabilities[0][i] <= 0.5 else 1
    
    return predictions

In [50]:
def model(X, t, learningRate=0.01, maxIters=1000):
    
    # number of samples
    M = X.shape[1]
    # number of features
    N = X.shape[0]
    
    w, b = initialize_parameters(N)
    costs = []
    
    for iteration in range(maxIters):
        
        grads, cost = propagate(X, w, b, t)
        w, b = update(w, b, grads, learningRate)
        costs.append(cost)
        
    return w, b, costs

In [85]:
def main():
    
    path = 'data/data.npy'
    data = load_data(path)
    
    # from here each sample is a row
    X, t = segregate_target(data)
    data = train_test_validation_split(X, t)
    
#     data['X_train'] = normalize(data['X_train'])
#     data['X_valid'] = normalize(data['X_valid'])
#     data['X_test'] = normalize(data['X_test'])
    
    # from here each sample is a column
    X_train = data['X_train'].T
    t_train = data['t_train'].reshape(1,-1)
    w, b, costs = model(X_train, t_train, learningRate=2.0, maxIters=50000)
    
    print(w, b)
    
    # print(costs[0], costs[-1])
    X_valid = data['X_valid'].T
    print(1 - np.sum(np.abs(predict(w, b, X_train).reshape(-1,1) - data['t_train'].reshape(-1,1))) / X_train.shape[1])
    print(1 - np.sum(np.abs(predict(w, b, X_valid).reshape(-1,1) - data['t_valid'].reshape(-1,1))) / X_valid.shape[1])

In [93]:
main()

KeyboardInterrupt: 

In [65]:
# path = 'data/data.npy'
# data = load_data(path)

# X, t = segregate_target(data)
# data = train_test_validation_split(X, t)

# data['X_train'] = normalize(data['X_train'])
# data['X_valid'] = normalize(data['X_valid'])
# data['X_test'] = normalize(data['X_test'])

# X_train = data['X_train']
# t_train = data['t_train']
# X_test = data['X_test']
# t_test = data['t_test']

# # print(X_train.shape)

# sklearn_model(X_train, np.ravel(t_train), X_test, np.ravel(t_test))

In [101]:
print(sigmoid(np.array([-15])))

[3.05902227e-07]


In [90]:
c = np.sum(np.array([1, 0, 1])*np.log(np.array([0.78358768, 0.66623071, 0.67362981])) + np.array([0, 1, 0])*np.log(np.array([1-0.78358768, 1-0.66623071, 1-0.67362981])))

In [91]:
print(c)

-1.7362521508103812
